In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%load_ext autoreload
%autoreload 2
from pyspark.sql import SQLContext
import boto3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from sklearn.metrics import confusion_matrix
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, HashingTF, IDF, CountVectorizer
from pyspark.ml.classification import LogisticRegression as spark_LogisticRegression
from pyspark.mllib.evaluation import MulticlassMetrics, BinaryClassificationMetrics
import cleaner

In [3]:
f_path = "s3://chizzy/yelp_data/"
df_bus = pd.read_json(f_path+"business.json", lines=True)
df_user = pd.read_json(f_path+"user.json", lines=True)
df_rev = pd.read_json(f_path+"review.json", lines=True)

In [4]:
df_bus.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,"4855 E Warner Rd, Ste B9","{'AcceptsInsurance': True, 'ByAppointmentOnly'...",FYWN1wneV18bWNgQjJ2GNg,"[Dentists, General Dentistry, Health & Medical...",Ahwatukee,"{'Friday': '7:30-17:00', 'Tuesday': '7:30-17:0...",1,33.330690,-111.978599,Dental by Design,,85044,22,4.0,AZ
1,3101 Washington Rd,"{'BusinessParking': {'garage': False, 'street'...",He-G7vWjzVUysIKrfNbPUQ,"[Hair Stylists, Hair Salons, Men's Hair Salons...",McMurray,"{'Monday': '9:00-20:00', 'Tuesday': '9:00-20:0...",1,40.291685,-80.104900,Stephen Szabo Salon,,15317,11,3.0,PA
2,"6025 N 27th Ave, Ste 1",{},KQPW8lFf1y5BT2MxiSZ3QA,"[Departments of Motor Vehicles, Public Service...",Phoenix,{},1,33.524903,-112.115310,Western Motor Vehicle,,85017,18,1.5,AZ
3,"5000 Arizona Mills Cr, Ste 435","{'BusinessAcceptsCreditCards': True, 'Restaura...",8DShNS-LuFqpEWIp0HxijA,"[Sporting Goods, Shopping]",Tempe,"{'Monday': '10:00-21:00', 'Tuesday': '10:00-21...",0,33.383147,-111.964725,Sports Authority,,85282,9,3.0,AZ
4,581 Howe Ave,"{'Alcohol': 'full_bar', 'HasTV': True, 'NoiseL...",PfOCPjBrlQAnz__NXj9h_w,"[American (New), Nightlife, Bars, Sandwiches, ...",Cuyahoga Falls,"{'Monday': '11:00-1:00', 'Tuesday': '11:00-1:0...",1,41.119535,-81.475690,Brick House Tavern + Tap,,44221,116,3.5,OH


In [5]:
df_user.head()

,average_stars,compliment_cool,compliment_cute,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,...,cool,elite,fans,friends,funny,name,review_count,useful,user_id,yelping_since
0,4.67,0,0,0,0,0,0,0,0,1,...,0,[],0,"[cvVMmlU1ouS3I5fhutaryQ, nj6UZ8tdGo8YJ9lUMTVWN...",0,Johnny,8,0,oMy_rEb0UBEmMlu-zcxnoQ,2014-11-03
1,3.70,0,0,0,0,0,0,0,0,0,...,0,[],0,"[0njfJmB-7n84DlIgUByCNw, rFn3Xe3RqHxRSxWOU19Gp...",0,Chris,10,0,JJ-aSuM4pCFPdkfoZ34q0Q,2013-09-24
2,2.00,0,0,0,0,0,0,0,0,0,...,0,[],0,[],0,Tiffy,1,0,uUzsFQn_6cXDh6rPNGbIFA,2017-03-02
3,4.67,0,0,0,0,0,0,0,0,0,...,0,[],0,[],0,Mark,6,0,mBneaEEH5EMyxaVyqS-72A,2015-03-13
4,4.67,0,0,0,0,0,0,0,0,0,...,0,[],0,[],0,Evelyn,3,0,W5mJGs-dcDWRGEhAzUYtoA,2016-09-08


In [6]:
df_rev.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,0W4lkclzZThpx3V65bVgig,0,2016-05-28,0,v0i_UHJMo_hPBq9bxWvW4w,5,"Love the staff, love the meat, love the place....",0,bv2nCi5Qv5vroFiqKGopiw
1,AEx2SYEUJmTxVVB18LlCwA,0,2016-05-28,0,vkVSCC7xljjrAI4UGfnKEQ,5,Super simple place but amazing nonetheless. It...,0,bv2nCi5Qv5vroFiqKGopiw
2,VR6GpWIda3SfvPC-lg9H3w,0,2016-05-28,0,n6QzIUObkYshz4dz2QRJTw,5,Small unassuming place that changes their menu...,0,bv2nCi5Qv5vroFiqKGopiw
3,CKC0-MOWMqoeWf6s-szl8g,0,2016-05-28,0,MV3CcKScW05u5LVfF6ok0g,5,Lester's is located in a beautiful neighborhoo...,0,bv2nCi5Qv5vroFiqKGopiw
4,ACFtxLv8pGrrxMm6EgjreA,0,2016-05-28,0,IXvOzsEMYtiJI0CARmj77Q,4,Love coming here. Yes the place always needs t...,0,bv2nCi5Qv5vroFiqKGopiw


In [7]:
#Rename colume names 
df_bus.rename(columns={"name":"bus_name","review_count":"bus_rev_count"}, inplace=True)
df_user.rename(columns={"name":"user_name", "review_count":"user_rev_count"}, inplace=True)
df_rev.rename(columns={"cool":"cool_rev", "funny":"funny_rev", "stars":"stars_rev", "useful":"useful_rev"}, inplace=True)

In [8]:
print("Number of rows for business table: {}".format(len(df_bus)))
print("Number of rows for user table: {}".format(len(df_user)))
print("Number of rows for review table: {}".format(len(df_rev)))

Number of rows for business table: 174567
Number of rows for user table: 1326101
Number of rows for review table: 5261669


In [9]:
us_states = ["AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA","KS", \
            "KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY", \
            "NC","ND","OH","OK","OR","PA","RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY"]

#Extracting reviews for only US restaurants
df_bus_us = df_bus[df_bus.state.isin (us_states)]

In [10]:
print("Number of rows for business table for US states: {}".format(len(df_bus_us)))

Number of rows for business table for US states: 128302


In [11]:
df_rev_bus = pd.merge(df_rev, df_bus, how='inner', on=['business_id', 'business_id'])

In [12]:
df_rev_bus.head()

,business_id,cool_rev,date,funny_rev,review_id,stars_rev,text,useful_rev,user_id,address,...,hours,is_open,latitude,longitude,bus_name,neighborhood,postal_code,bus_rev_count,stars,state
0,0W4lkclzZThpx3V65bVgig,0,2016-05-28,0,v0i_UHJMo_hPBq9bxWvW4w,5,"Love the staff, love the meat, love the place....",0,bv2nCi5Qv5vroFiqKGopiw,3895 Boulevard Saint-Laurent,...,"{'Monday': '8:00-0:30', 'Tuesday': '8:00-0:30'...",1,45.516373,-73.577537,Schwartz's,Plateau-Mont-Royal,H2W 1X9,1953,4.0,QC
1,0W4lkclzZThpx3V65bVgig,0,2011-09-28,0,FunI9om-aK5oMIIJmhMlZA,4,a must stop when you are in montreal!\n\nI was...,0,u0LXt3Uea_GidxRW1xcsfg,3895 Boulevard Saint-Laurent,...,"{'Monday': '8:00-0:30', 'Tuesday': '8:00-0:30'...",1,45.516373,-73.577537,Schwartz's,Plateau-Mont-Royal,H2W 1X9,1953,4.0,QC
2,0W4lkclzZThpx3V65bVgig,2,2016-02-22,1,ufI413zxZqGwIBIInnM5Rg,4,*Schwartz's has become a Montreal's famous lan...,4,xYciRtVZ1PW4IxSX4oJ1aw,3895 Boulevard Saint-Laurent,...,"{'Monday': '8:00-0:30', 'Tuesday': '8:00-0:30'...",1,45.516373,-73.577537,Schwartz's,Plateau-Mont-Royal,H2W 1X9,1953,4.0,QC
3,0W4lkclzZThpx3V65bVgig,0,2013-07-01,0,cDUhgPLuLR8hQDNgkGh_hw,4,"Boss: ""You want to go to a conference in Mont...",0,ayaxadJdy0ObFKaMlgfNVw,3895 Boulevard Saint-Laurent,...,"{'Monday': '8:00-0:30', 'Tuesday': '8:00-0:30'...",1,45.516373,-73.577537,Schwartz's,Plateau-Mont-Royal,H2W 1X9,1953,4.0,QC
4,0W4lkclzZThpx3V65bVgig,0,2015-12-22,0,gE2sCFSGf8bhZ07uYRjpnw,4,I went to Montreal a few weeks ago Schwartz wa...,0,aU4ba7OlYKOKJ9kpvbOcsg,3895 Boulevard Saint-Laurent,...,"{'Monday': '8:00-0:30', 'Tuesday': '8:00-0:30'...",1,45.516373,-73.577537,Schwartz's,Plateau-Mont-Royal,H2W 1X9,1953,4.0,QC


In [13]:
df_rev_bus_user = pd.merge(df_rev_bus, df_user, how='left', on=['user_id', 'user_id'])

In [14]:
print("Number of rows for business table for concated business, user and review: {}".format(len(df_rev_bus_user)))

Number of rows for business table for concated business, user and review: 5261669


In [15]:
df_rev_bus_user.head()

,business_id,cool_rev,date,funny_rev,review_id,stars_rev,text,useful_rev,user_id,address,...,compliment_writer,cool,elite,fans,friends,funny,user_name,user_rev_count,useful,yelping_since
0,0W4lkclzZThpx3V65bVgig,0,2016-05-28,0,v0i_UHJMo_hPBq9bxWvW4w,5,"Love the staff, love the meat, love the place....",0,bv2nCi5Qv5vroFiqKGopiw,3895 Boulevard Saint-Laurent,...,0,0,[],0,[4Zpn3ObC2zexkIhor4LJIQ],0,Tim,6,0,2015-12-22
1,0W4lkclzZThpx3V65bVgig,0,2011-09-28,0,FunI9om-aK5oMIIJmhMlZA,4,a must stop when you are in montreal!\n\nI was...,0,u0LXt3Uea_GidxRW1xcsfg,3895 Boulevard Saint-Laurent,...,3,22,"[2012, 2014, 2016, 2017, 2015, 2013]",8,"[QSpRpbemKx5U4-NtUumUbw, G-6X-llgA_qAxGxocykHz...",7,Julia,359,79,2009-08-20
2,0W4lkclzZThpx3V65bVgig,2,2016-02-22,1,ufI413zxZqGwIBIInnM5Rg,4,*Schwartz's has become a Montreal's famous lan...,4,xYciRtVZ1PW4IxSX4oJ1aw,3895 Boulevard Saint-Laurent,...,8,0,"[2015, 2017, 2016]",30,"[UX0Lh2n7gOiH5PVsCo1psA, GM7NRYBXxl3ELf8y4Loq0...",14,Anh,177,39,2015-02-15
3,0W4lkclzZThpx3V65bVgig,0,2013-07-01,0,cDUhgPLuLR8hQDNgkGh_hw,4,"Boss: ""You want to go to a conference in Mont...",0,ayaxadJdy0ObFKaMlgfNVw,3895 Boulevard Saint-Laurent,...,0,0,[],1,"[_V28Y9fkcQgo3-F24DDxTA, ZCIvJM3fuFYpUFIG4OFzj...",0,Tuan,26,1,2012-02-03
4,0W4lkclzZThpx3V65bVgig,0,2015-12-22,0,gE2sCFSGf8bhZ07uYRjpnw,4,I went to Montreal a few weeks ago Schwartz wa...,0,aU4ba7OlYKOKJ9kpvbOcsg,3895 Boulevard Saint-Laurent,...,1,6,[2017],1,"[PhJhg24zecO2USi_blq6eg, uOQTLqNUtiuNWxaTma02E...",3,Kenneth,32,10,2015-08-15


In [16]:
df_rev_bus_user.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5261669 entries, 0 to 5261668
Data columns (total 44 columns):
business_id           object
cool_rev              int64
date                  datetime64[ns]
funny_rev             int64
review_id             object
stars_rev             int64
text                  object
useful_rev            int64
user_id               object
address               object
attributes            object
categories            object
city                  object
hours                 object
is_open               int64
latitude              float64
longitude             float64
bus_name              object
neighborhood          object
postal_code           object
bus_rev_count         int64
stars                 float64
state                 object
average_stars         float64
compliment_cool       int64
compliment_cute       int64
compliment_funny      int64
compliment_hot        int64
compliment_list       int64
compliment_more       int64
compliment_note  

In [25]:
df = df_rev_bus_user[["text", "stars_rev"]][df_rev_bus_user["state"]=="WI"]
df.head()

,text,stars_rev
13480,We got a permit to have wedding photos taken h...,4
13481,Great place any time of the year. The family a...,4
13482,"Gorgeous, peaceful, relaxing, but still being ...",5
13483,"Free and much larger than I expected, this bot...",5
13484,Fantastic - so many things to see in the conse...,5


In [32]:
df1 = df.head(80000)
corpus = df1["text"].values

In [33]:
print("Number of words reviews in corpus: {}".format(len(corpus)))

Number of words reviews in corpus: 80000


In [34]:
corpus[3]

'Free and much larger than I expected, this botanical garden was a pleasant surprise.  There are a few corners that are particularly nice, like the Rose Tower and Temple. Meander throughout and try to get lost. Sit a spell on a bench and enjoy the scenery.'

In [35]:
print("Number of words befor cleaning, and removing stopwords: {}".format(len(corpus[3].split())))

Number of words befor cleaning, and removing stopwords: 46


In [36]:
# nltk.download('stopwords')
# nltk.download('wordnet')

In [37]:
sw = set(stopwords.words("english"))
tokenizer = RegexpTokenizer("[\w']+")
st = PorterStemmer()
lemma = WordNetLemmatizer()

In [38]:
cleaned = cleaner.clean_stem(corpus, tokenizer, lemma, sw)

In [39]:
cleaned[3]

'free much larger i expected botanical garden pleasant surprise there corner particularly nice like rose tower temple meander throughout try get lost sit spell bench enjoy scenery'

In [40]:
print("Number of words befor cleaning, and removing stopwords: {}".format(len(corpus[3].split())))
print("Number of words after cleaning, and removing stopwords: {}".format(len(cleaned[3].split())))

Number of words befor cleaning, and removing stopwords: 46
Number of words after cleaning, and removing stopwords: 27


In [41]:
tfidf_vectorizer= TfidfVectorizer(max_df=0.95, min_df=2,
                                stop_words='english')

tfidf = tfidf_vectorizer.fit_transform(cleaned)

In [42]:
X = tfidf.toarray()
X.shape

(80000, 28978)

In [43]:
y = df1["stars_rev"].values

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

### LogisticRegression Linear model

In [45]:
lreg = LogisticRegression()
lreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [46]:
y_pred = lreg.predict(X_test)

In [47]:
def confu(y_test, y_pred):
    tp, tn, fp, fn = 0,0,0,0
    for x,y in zip(y_test, y_pred):
        if x == y:
            if x > 3:
                tp+=1
            else:
                tn+=1
        elif x != y:
            if y>3:
                fp+=1
            else:
                fn+1
    return tp, tn, fp, fn

In [48]:
def metrics(y_test, y_pred):
    tp, tn, fp, fn = confu(y_test, y_pred)
    matrix = np.array([[tp, fp], [fn, tn]])
    recall = tp/(tp+fn)
    precision = tp/(fp+tp)
    accuracy = (tp+tn)/(tn+fp+fn+tp)
    return matrix, recall, precision, accuracy

In [49]:
matrix, recall, precision, accuracy = metrics(y_test, y_pred)

In [50]:
#confusion matrix
matrix

array([[9115, 6260],
       [   0, 2590]])

In [51]:
print("Recall: {}%".format(round(recall*100, 2)))
print("Precision: {}%".format(round(precision*100, 2)))
print("Accuracy: {}%".format(round(accuracy*100, 2)))

Recall: 100.0%
Precision: 59.28%
Accuracy: 65.15%


In [52]:
terms = tfidf_vectorizer.get_feature_names()
pos_terms, neg_terms = cleaner.show_topics(lreg.coef_, terms, length=40)

In [53]:
pos_terms

[('amazing', 6.427942258289969),
 ('best', 5.940899175594272),
 ('favorite', 5.225930672287809),
 ('highly', 4.61118355861173),
 ('perfect', 4.187692709884128),
 ('fantastic', 4.034304243498077),
 ('delicious', 3.8282386881612456),
 ('awesome', 3.77864442101737),
 ('thank', 3.7623172930455118),
 ('excellent', 3.739485689433928),
 ('love', 3.553912356995891),
 ('incredible', 3.5101015711007797),
 ('madison', 3.392557784277208),
 ('great', 3.3039965405538205),
 ('outstanding', 3.101611181426327),
 ('wonderful', 3.0717821909025607),
 ('gem', 3.038808779711679),
 ('phenomenal', 3.0098764832619653),
 ('dr', 2.936947897170193),
 ('professional', 2.885177302806769),
 ('heaven', 2.870526731978165),
 ('wow', 2.864384394292777),
 ('die', 2.820399711760053),
 ('knowledgeable', 2.685394485942716),
 ('dream', 2.6053422610918773),
 ('perfection', 2.584821627916357),
 ('superb', 2.5388899701288707),
 ('absolutely', 2.517741209130546),
 ('perfectly', 2.5173916529867735),
 ('notch', 2.501765118833812),

In [54]:
neg_terms

[('ok', -6.142233414961973),
 ('worst', -5.720726519883465),
 ('okay', -5.39196989139932),
 ('mediocre', -5.074060685240222),
 ('decent', -5.040026145832152),
 ('rude', -4.973554306325695),
 ('bland', -4.890473133803314),
 ('terrible', -4.594662761234788),
 ('disappointing', -4.592928469428883),
 ('overall', -4.469694441577718),
 ('horrible', -4.410153334269946),
 ('overpriced', -4.377353283986161),
 ('average', -4.223699600333083),
 ('pretty', -4.1676874940039745),
 ('slow', -4.058221763181025),
 ('awful', -3.869442366034459),
 ('meh', -3.742452235362794),
 ('unfortunately', -3.7386252288467605),
 ('solid', -3.5234174898229824),
 ('guess', -3.4576773094918094),
 ('lacked', -3.386422336328378),
 ('lack', -3.115328375863269),
 ('alright', -3.0745227072170156),
 ('poor', -3.0427716008741457),
 ('bit', -2.9868434291817203),
 ('reason', -2.915646846666509),
 ('bad', -2.798018153744896),
 ('gross', -2.7903472691125937),
 ('said', -2.777490273136886),
 ('asked', -2.726289445731365),
 ('lacki